In [ ]:
from pathlib import Path

import deepecohab
import toml

#### Make project

When using custom layout we need to provide a dictionary that will rename the antennas so that no antenna number is repeated in the data. Then, after we created the project, we specify correct antenna pairs and positions. 

<span style="color:yellow"><b>!!!IMPORTANT!!!</b></span>

Keep in mind that in the traditional layout we interpolate animal position. 

It means that when one antenna on the trajectory is missing we assign the position to be the cage that is on the way. So if the pair of antennas is `1` - `3` and antenna `2` is missing, we assign it to position `"cage_2"`. 

This allows for better approximation of time spent in cages, as in general time spent in tunnels is relatively short in each visit.

In [ ]:
rename_dict = {
	"COM4": {
		"3": 9,
		"4": 10,
		"5": 1,
		"6": 2,
		"7": 3,
		"8": 4,
	},
	"COM5": {
		"3": 11,
		"4": 12,
		"5": 5,
		"6": 6,
		"7": 7,
		"8": 8,
	},
}

example_dir = Path.cwd()

In [ ]:
config_path = deepecohab.create_ecohab_project(
	project_location=example_dir,  # Location where the project will be created
	experiment_name="test_custom_name2",  # Name of the project
	data_path=example_dir / "test_data_custom",  # Folder with ecohab txt files
	light_phase_start="00:00:00",
	dark_phase_start="12:00:00",
	custom_layout=True,
	antenna_rename_scheme=rename_dict,
)

#### Modify the config `antenna_combinations`

This image represents the layout of the cages and antennas:

<img src="../examples\test_data_custom\img.svg" alt="drawing" width="800"/>

For you convenience we prepared the correct antenna combinations for this layout taking this interpolation of position into account

In [ ]:
# Load config as a dict
cfg = toml.load(config_path)

# Create new antenna_combinations dict
cfg["antenna_combinations"] = {
	"9_9": "cage_s1",
	"11_11": "cage_s2",
	"10_9": "c1_s1",
	"9_10": "s1_c1",
	"1_2": "c1_c2",
	"2_1": "c2_c1",
	"3_4": "c2_c3",
	"4_3": "c3_c2",
	"5_6": "c3_c4",
	"6_5": "c4_c3",
	"7_8": "c4_c1",
	"8_7": "c1_c4",
	"12_11": "c3_s2",
	"11_12": "s2_c3",
	"8_10": "cage_1",
	"10_8": "cage_1",
	"1_10": "cage_1",
	"10_1": "cage_1",
	"8_1": "cage_1",
	"1_8": "cage_1",
	"7_10": "cage_1",
	"10_7": "cage_1",
	"8_9": "cage_1",
	"9_8": "cage_1",
	"8_2": "cage_1",
	"2_8": "cage_1",
	"7_1": "cage_1",
	"1_7": "cage_1",
	"10_2": "cage_1",
	"2_10": "cage_1",
	"1_9": "cage_1",
	"9_1": "cage_1",
	"1_3": "cage_2",
	"3_1": "cage_2",
	"2_4": "cage_2",
	"4_2": "cage_2",
	"2_3": "cage_2",
	"3_2": "cage_2",
	"3_5": "cage_3",
	"5_3": "cage_3",
	"3_12": "cage_3",
	"12_3": "cage_3",
	"6_4": "cage_3",
	"4_6": "cage_3",
	"6_12": "cage_3",
	"12_6": "cage_3",
	"11_4": "cage_3",
	"4_11": "cage_3",
	"11_5": "cage_3",
	"5_11": "cage_3",
	"4_5": "cage_3",
	"5_4": "cage_3",
	"4_12": "cage_3",
	"12_4": "cage_3",
	"5_12": "cage_3",
	"12_5": "cage_3",
	"8_6": "cage_4",
	"6_8": "cage_4",
	"7_5": "cage_4",
	"5_7": "cage_4",
	"7_6": "cage_4",
	"6_7": "cage_4",
}

cfg["tunnels"] = {
	"c1_c2": "tunnel_1",
	"c2_c1": "tunnel_1",
	"c2_c3": "tunnel_2",
	"c3_c2": "tunnel_2",
	"c3_c4": "tunnel_3",
	"c4_c3": "tunnel_3",
	"c4_c1": "tunnel_4",
	"c1_c4": "tunnel_4",
	"s1_c1": "tunnel_s1",
	"c1_s1": "tunnel_s1",
	"s2_c3": "tunnel_s2",
	"c3_s2": "tunnel_s2",
}

# Save the updated config
with open(config_path, "w") as f:
	toml.dump(cfg, f)

#### Make structure

In [ ]:
df = deepecohab.get_ecohab_data_structure(
	config_path,
	custom_layout=True,
)

#### Chasings and ranking

In [ ]:
chasings = deepecohab.calculate_chasings(config_path)
ranking_ordinal = deepecohab.calculate_ranking(config_path)

#### Activity

In [ ]:
activity = deepecohab.calculate_activity(config_path)
cage_occupancy = deepecohab.calculate_cage_occupancy(config_path)

#### Sociability

In [ ]:
pairwise_meetings = deepecohab.calculate_pairwise_meetings(config_path, minimum_time=2)
time_alone = deepecohab.calculate_time_alone(config_path)
in_cohort_sociability = deepecohab.calculate_incohort_sociability(config_path)

#### Dahsboard

In [ ]:
deepecohab.run_dashboard(config_path)